#SHAWNEE STATE PARK RUCKING HEATMAP

###This code will utilize a bulk export of my Strava-recorded rucks this summer in the form of gpx files to create a heatmap. 

####Import Modules

In [18]:
import gpxpy
import os
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap, HeatMapWithTime
from folium.features import DivIcon
from datetime import datetime
import json
import numpy as np

####Set Up Directories

In [3]:
#Get list of gpx files in Strava activities folder
dir = os.path.expanduser("~/Desktop/Strava_Export/activities/")
files = os.listdir(dir)

#Get json of Shawnee State Park boundary -- downloaded from https://www.pasda.psu.edu/
ssp_file = os.path.expanduser("~/Desktop/shawnee_statepark.geojson")
sfile = json.load(open(ssp_file))

####Create Dataframe and Tuple from GPX Files

In [4]:
#Create empty tuple for heatmap with time
points2 = []

#Define function to parse gpx files
def parsegpx(f):
    points = []
    with open(os.path.join(dir,f), 'r') as gpxfile:
        #gpx = gpxpy.parse(gpxfile)
        for track in gpxpy.parse(gpxfile).tracks:
            for segment in track.segments:
                for point in segment.points:
                    dict = {"Timestamp" : point.time,
                           "Latitude" : point.latitude,
                           "Longitude" : point.longitude,
                           "Elevation" : point.elevation}
                    points.append(dict)
                    points2.append(tuple([point.latitude, point.longitude]))
    return points

#Create dataframe of gps points
df2 = pd.concat([pd.DataFrame(parsegpx(f)) for f in files], keys = files)


####Calculate Sum of Distance Rucked

In [5]:
#Function to calculate distance between two coordinates
def haversine(lat1, lon1, lat2, lon2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

distsum = 0

#Calculate sum of distances of individual rucks 
for i in range(len(files)):
    distd = df2.loc[files[i], ("Latitude", "Longitude")]
    
    distd['dist'] = \
    haversine(distd.Latitude.shift(), distd.Longitude.shift(),
                 distd.loc[1:, 'Latitude'], distd.loc[1:, 'Longitude'])

    distsum = distsum + distd['dist'].sum()
    
#Convert to miles
distsum = distsum/1.609



####Create New Dataframe to Calculate Total Elevation Gain

In [9]:
df3 = df2.copy()

#Modify dataframe for grouping by date of ruck
df3['Timestamp'] = df3['Timestamp'].apply(lambda x: x.replace(minute=0,hour=0))

#Calculate elevation gain in meters
df3 = (df3.groupby('Timestamp').apply(lambda x: x[['Elevation']].values.max().tolist()) - df3.groupby('Timestamp').apply(lambda x: x[['Elevation']].values.min().tolist()))

#Convert sum to feet
elevation_gain = df3.sum() * 3.28084



####Create Dataframe and List Necessary for Heatmap with Time

In [8]:
df2['Timestamp'] = df2['Timestamp'].apply(lambda x: x.replace(microsecond=0,second=0))
df2['Weight'] = 1
map_data = df2.groupby('Timestamp').apply(lambda x: x[['Latitude', 'Longitude', 'Weight']].values.tolist())
date_index = [x.strftime('%m/%d/%Y, %H:%M:%S') for x in map_data.index]

####Create Map Using Folium Module

In [33]:
#Create map for ruck 
ruckmap = folium.Map(location=[(df2.Latitude.mean()-.001), (df2.Longitude.mean()+.005)], zoom_start=14, tiles='cartodbdark_matter')

#Add Shawnee State Park geojson to map
shawnee_boundary = folium.GeoJson(
    sfile,
    tooltip='Shawnee State Park',
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': '#2e4d24',
        'opacity': 1,
        'weight': '2'
    }
)
shawnee_boundary.add_to(ruckmap)

#Create heatmap with time layer to animate individual rucks
ruckmap_data = map_data.tolist()
hmt = HeatMapWithTime(ruckmap_data,index=date_index,min_opacity=60,radius=7, min_speed=5, gradient={.1:'white', .3:'lightblue'}, auto_play=True)

#Create heatmap layer of all rucks
hm = HeatMap(points2, min_opacity=0, blur=5, max_val=10, radius=4, gradient={.2: '#FACBFB', .3: 'purple', .5: 'red', .7:'yellow'})

#Add heatmap layers to ruckmap
hm.add_to(ruckmap)
hmt.add_to(ruckmap)

#Add static ruck stats to map
folium.map.Marker(
    [40.046, -78.595],
    icon=DivIcon(
        icon_size=(220,200),
        icon_anchor=(0,40),
        html='<div style="font-size: 11pt; background-color: #9c9b98; text-align: center">Total Elevation Gain: {0} ft\nTotal Distance: {1} miles</div>'.format(round(elevation_gain, 0), round(distsum, 1))
        )
    ).add_to(ruckmap)
ruckmap

####To View Map Visit
https://brycestouffer.github.io/

####Save map to local machine git repo

In [34]:
#outdir = os.path.expanduser('~/Desktop/Strava_Export/')
#ruckmap.save(os.path.join(outdir, 'ruckmap_heatmap.html'))
outdirgit = os.path.expanduser('~/Sites/brycestouffer.github.io/')
ruckmap.save(os.path.join(outdirgit, 'index.html'))

###Useful Resources
####GPX Parsing
https://www.ryanbaumann.com/blog/2015/7/18/strava-heat-maps-part-2
####Folium Heatmap Creation
https://towardsdatascience.com/build-interactive-gps-activity-maps-from-gpx-files-using-folium-cf9eebba1fe7
####Folium Heatmap with Time
https://www.coursera.org/lecture/machine-learning-asset-management-alternative-data/lab-session-mapping-data-with-folium-eG29Q
####Distance Calculation
https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas/29546836#29546836
####Shawnee State Park GeoJSON
https://www.pasda.psu.edu/uci/DataSummary.aspx?dataset=114